In [ ]:
import numpy as np
import imageio as iio
import matplotlib.pyplot as plt
import taichi as ti
ti.init(arch=ti.gpu)
GUI_DOWNSCALING_FACTOR = 1.0
ZOOM_WIDTH = 200.0

In [ ]:
# FILENAME = 'B0_dn'
# FILENAME = 'B1_dn'
# FILENAME = 'B2_dn'
# FILENAME = 'B3_dn'
# FILENAME = 'T0_dn'
# FILENAME = 'T1_dn'
# FILENAME = 'T2_dn'
# FILENAME = 'T3_dn'

FILENAME = 'frame25'

image = iio.imread(FILENAME + '.pfm', format='PFM-FI')
image.shape, image.dtype

In [ ]:
image_grid = ti.field(dtype=ti.f32, shape=image.shape)
image_grid_downsampled = ti.field(dtype=ti.f32,\
    shape=(int(image.shape[1]/GUI_DOWNSCALING_FACTOR), int(image.shape[0]/GUI_DOWNSCALING_FACTOR), image.shape[2]))
igaussian_grid_A = ti.field(dtype=ti.f32, shape=image.shape)
igaussian_grid_B = ti.field(dtype=ti.f32, shape=image.shape)

In [ ]:
@ti.kernel
def copy(dst: ti.template(), src: ti.template()):
    for p in ti.grouped(dst):
        dst[p] = src[p]

@ti.kernel
def tonemap_hdr(exposure: ti.f32, gamma: ti.f32, i_red: ti.f32, i_green: ti.f32, i_blue: ti.f32):
    for x, y in ti.ndrange(image_grid.shape[0], image_grid.shape[1]):
        image_grid[x, y, 0] = (1.0 - ti.exp(-exposure * i_red   * image_grid[x, y, 0])) ** (1.0/gamma)
        image_grid[x, y, 1] = (1.0 - ti.exp(-exposure * i_green * image_grid[x, y, 1])) ** (1.0/gamma)
        image_grid[x, y, 2] = (1.0 - ti.exp(-exposure * i_blue  * image_grid[x, y, 2])) ** (1.0/gamma)
        
KERNEL = [5.0, 1.0, 1.0/1.41]
KERNEL_NORM = KERNEL[0] + 4.0 * KERNEL[1] + 4.0 * KERNEL[2]

@ti.kernel
def incremental_blur():
    for x, y, ch in igaussian_grid_B:
        value = KERNEL[0] * igaussian_grid_A[x, y, ch]\
              + KERNEL[1] * igaussian_grid_A[x-1, y, ch]\
              + KERNEL[1] * igaussian_grid_A[x+1, y, ch]\
              + KERNEL[1] * igaussian_grid_A[x, y-1, ch]\
              + KERNEL[1] * igaussian_grid_A[x, y+1, ch]\
              + KERNEL[2] * igaussian_grid_A[x-1, y-1, ch]\
              + KERNEL[2] * igaussian_grid_A[x+1, y+1, ch]\
              + KERNEL[2] * igaussian_grid_A[x+1, y-1, ch]\
              + KERNEL[2] * igaussian_grid_A[x-1, y+1, ch]
        igaussian_grid_B[x, y, ch] = value / KERNEL_NORM
    for x, y, ch in igaussian_grid_A:
        value = KERNEL[0] * igaussian_grid_B[x, y, ch]\
              + KERNEL[1] * igaussian_grid_B[x-1, y, ch]\
              + KERNEL[1] * igaussian_grid_B[x+1, y, ch]\
              + KERNEL[1] * igaussian_grid_B[x, y-1, ch]\
              + KERNEL[1] * igaussian_grid_B[x, y+1, ch]\
              + KERNEL[2] * igaussian_grid_B[x-1, y-1, ch]\
              + KERNEL[2] * igaussian_grid_B[x+1, y+1, ch]\
              + KERNEL[2] * igaussian_grid_B[x+1, y-1, ch]\
              + KERNEL[2] * igaussian_grid_B[x-1, y+1, ch]
        igaussian_grid_A[x, y, ch] = value / KERNEL_NORM
        
@ti.kernel
def unsharp_compose(intensity: ti.f32):
    for x, y, ch in image_grid:
        base = image_grid[x, y, ch]
        blurred = igaussian_grid_A[x, y, ch]
        image_grid[x, y, ch] = max(0.0, min(base + intensity * (base - blurred), 1.0))
        
@ti.kernel
def img_downsample():
    for x, y, ch in image_grid_downsampled:
        image_grid_downsampled[x, y, ch] = image_grid[int((image_grid.shape[0]-GUI_DOWNSCALING_FACTOR*y)), int(GUI_DOWNSCALING_FACTOR*x), ch]
        
@ti.kernel
def img_downsample_with_zoom(zoom_x: ti.f32, zoom_y: ti.f32, zoom_r: ti.f32):
    for x, y, ch in image_grid_downsampled:
        if x > zoom_x - zoom_r and x < zoom_x + zoom_r and y > zoom_y - zoom_r and y < zoom_y + zoom_r:
            image_grid_downsampled[x, y, ch] = image_grid[\
                            int(image_grid.shape[0]-GUI_DOWNSCALING_FACTOR*zoom_y) - (y-zoom_y),\
                            int(GUI_DOWNSCALING_FACTOR*zoom_x) - (zoom_x - x), ch]
        else:
            image_grid_downsampled[x, y, ch] = image_grid[int((image_grid.shape[0]-GUI_DOWNSCALING_FACTOR*y)), int(GUI_DOWNSCALING_FACTOR*x), ch]

In [ ]:
gui = ti.GUI('Tonemap HDR', (image_grid_downsampled.shape[0], image_grid_downsampled.shape[1]))
exposure = gui.slider('exposure', 0.0, 10.0, 0.01); exposure.value = 2.0 # 2.55
gamma = gui.slider('gamma', 0.1, 5.0, 0.01); gamma.value = 1.6 # 1.55
scale = gui.slider('unsharp scale', 0.0, 30.0, 1.0); scale.value = 8.0
sharpness = gui.slider('unsharp intensity', 0.0, 2.0, 0.01); sharpness.value = 0.3
i_red = gui.slider('I-red', 0.0, 2.0, 0.01); i_red.value = 1.1
i_green = gui.slider('I-green', 0.0, 2.0, 0.01); i_green.value = 1.05
i_blue = gui.slider('I-blue', 0.0, 2.0, 0.01); i_blue.value = 1.0
while gui.running:
    do_zoom = False
    do_mega_zoom = False
    zoom_x, zoom_y = gui.get_cursor_pos()
    gui.get_event()
    if gui.is_pressed(ti.GUI.RMB):
        do_zoom = True
        zoom_x, zoom_y = gui.get_cursor_pos()
    if gui.is_pressed(ti.GUI.MMB):
        do_mega_zoom = True

    image_grid.from_numpy(image)
    tonemap_hdr(exposure.value, gamma.value, i_red.value, i_green.value, i_blue.value)
    
    copy(igaussian_grid_A, image_grid)
    for n in range(int(scale.value)):
        incremental_blur()
    unsharp_compose(sharpness.value)
    
    if do_zoom:
        img_downsample_with_zoom(\
                int(zoom_x * image_grid_downsampled.shape[0]),\
                int(zoom_y * image_grid_downsampled.shape[1]), ZOOM_WIDTH)
    elif do_mega_zoom:
        img_downsample_with_zoom(\
                int(zoom_x * image_grid_downsampled.shape[0]),\
                int(zoom_y * image_grid_downsampled.shape[1]), 5.0 * ZOOM_WIDTH)
    else:
        img_downsample()

    gui.set_image(image_grid_downsampled)
    gui.show();

In [ ]:
image_final = image_grid.to_numpy()
plt.figure(figsize = (10.0, 10.0))
plt.imshow(image_final);

In [ ]:
iio.imwrite(FILENAME + '.exr', image_final, format='EXR-FI')
image_final_LDR = 255.0 * image_final
iio.imwrite(FILENAME + '.png', image_final_LDR.astype(np.uint8), format='PNG-FI')